## Tensorflow HuBMAP - Hacking the Kidney competition starter kit:
* https://www.kaggle.com/wrrosa/hubmap-tf-with-tpu-efficientunet-512x512-tfrecs (how to create training and inference tfrecords)
* https://www.kaggle.com/wrrosa/hubmap-tf-with-tpu-efficientunet-512x512-train (training pipeline)
* this notebook (inference with submission)

# Versions
* V1 (V7 train notebook) 4-CV efficientunetb0 512x512 (**LB .834**)
* V2 (V8 train notebook) loss bce (LB .835)
* V3 (V9 train notebook) efficientunetb1 (CV .871, LB .830)
* V4 (V10 train notebook) efficientunetb4 (CV .874, **LB .839**)
* V5 (V12 train notebook) efficientunetb7 (CV .858, LB .835)
* V6 (V13 train notebook) efficientunetb4 (CV .877, LB .836)
* V7 (V14 train notebook) efficientunetb4 with overlapped train data, summing preds in inference (CV .879, **LB .843**)
* V8 (V14 train notebook) efficientunetb4  THRESHOLD=0.4, interpolation = cv2.INTER_AREA, rle_encode_less_memory (**LB .846**)
* V9 (V14 train notebook) efficientunetb4, MIN_OVERLAP = 300 (**LB 0.848**)
* V10 (V14 train notebook) efficientunetb4, checksum mask before modifications (1h 11m, no need to score)
* V11 (V14 train notebook) efficientunetb4, SUBMISSION_MODE added (generate submission from public tfrec files, almost 20m = 3.5 times faster!)
* V12 (V14 train notebook) efficientunetb4, CHECKSUM = False (**LB 0.848**)
* V13 (V15 train notebook) efficientunetb4, (**LB 0.849**)
* V14 (V15 train notebook) efficientunetb4, switch public tfrec files path: from nb output to dataset (**LB 0.849**)
* V15 (V15 train notebook) efficientunetb4, competition data update, model not updated  (**LB 0.903**)
* V16 (V15 train notebook) efficientunetb4, fixed issue with new data and FULL mode, full submission took about 3hrs compared to 1hr submission on public test set. It looks like private test set is 2 times larger then public. (**LB 0.903**)
* V17 (V15 train notebook) efficientunetb4, back to PUBLIC_TFREC mode without CHECKSUM, with THRESHOLD = 0.5 (**LB 0.897**)
* V18 (V16 train notebook) efficientunetb4, model trained with new competition data (**LB 0.890**)
* V19 (V16 train notebook) efficientunetb4, THRESHOLD = 0.4 (...)

# Refferences:
* https://www.kaggle.com/joshi98kishan/hubmap-keras-pipeline-training-inference
* https://www.kaggle.com/bguberfain/memory-aware-rle-encoding/
* https://www.kaggle.com/leighplt/pytorch-fcn-resnet50
* https://www.kaggle.com/c/hubmap-kidney-segmentation/discussion/224883#1233186

# Parameters
Read parameteres from notebook output, actually only **DIM** is used:

In [ ]:
mod_path = '/kaggle/input/hubmap-tf-with-tpu-efficientunet-512x512-train/'
import yaml
import pprint
with open(mod_path+'params.yaml') as file:
    P = yaml.load(file, Loader=yaml.FullLoader)
    pprint.pprint(P)
    
THRESHOLD = 0.4 # preds > THRESHOLD
WINDOW = 1024
MIN_OVERLAP = 300
NEW_SIZE = P['DIM']
 
SUBMISSION_MODE = 'FULL' # PUBLIC_TFREC or FULL
# 'PUBLIC_TFREC' = use created tfrecords for public test set with MIN_OVERLAP = 300 tiling 1024-512, ignore other (private test) data
# 'FULL' do not use tfrecords, just full submission 

CHECKSUM = False # compute mask sum for each image


# Metrics

In [ ]:
import json

with open(mod_path + 'metrics.json') as json_file:
    M = json.load(json_file)
print('Model run datetime: '+M['datetime'])
print('OOF val_dice_coe: ' + str(M['oof_dice_coe']))

# Packages

In [ ]:
! pip install ../input/kerasapplications/keras-team-keras-applications-3b180cb -f ./ --no-index -q
! pip install ../input/efficientnet/efficientnet-1.1.0/ -f ./ --no-index -q
import numpy as np
import pandas as pd
import os
import glob
import gc

import rasterio
from rasterio.windows import Window

import pathlib
from tqdm.notebook import tqdm
import cv2

import tensorflow as tf
import efficientnet as efn
import efficientnet.tfkeras

# Functions

In [ ]:
def rle_encode_less_memory(img):
    pixels = img.T.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def make_grid(shape, window=256, min_overlap=32):
    """
        Return Array of size (N,4), where N - number of tiles,
        2nd axis represente slices: x1,x2,y1,y2 
    """
    x, y = shape
    nx = x // (window - min_overlap) + 1
    x1 = np.linspace(0, x, num=nx, endpoint=False, dtype=np.int64)
    x1[-1] = x - window
    x2 = (x1 + window).clip(0, x)
    ny = y // (window - min_overlap) + 1
    y1 = np.linspace(0, y, num=ny, endpoint=False, dtype=np.int64)
    y1[-1] = y - window
    y2 = (y1 + window).clip(0, y)
    slices = np.zeros((nx,ny, 4), dtype=np.int64)
    
    for i in range(nx):
        for j in range(ny):
            slices[i,j] = x1[i], x2[i], y1[j], y2[j]    
    return slices.reshape(nx*ny,4), [x1, x2, y1, y2]

# Models

In [ ]:
identity = rasterio.Affine(1, 0, 0, 0, 1, 0)
fold_models = []
for fold_model_path in glob.glob(mod_path+'*.h5'):
    fold_models.append(tf.keras.models.load_model(fold_model_path,compile = False))
print(len(fold_models))

# Tfrecords functions

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
image_feature = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'x1': tf.io.FixedLenFeature([], tf.int64),
    'y1': tf.io.FixedLenFeature([], tf.int64)
}
def _parse_image(example_proto):
    example = tf.io.parse_single_example(example_proto, image_feature)
    image = tf.reshape( tf.io.decode_raw(example['image'],out_type=np.dtype('uint8')), (P['DIM'],P['DIM'], 3))
    return image, example['x1'], example['y1']

def load_dataset(filenames, ordered=True):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(_parse_image)
    return dataset

def get_dataset(FILENAME):
    dataset = load_dataset(FILENAME)
    dataset  = dataset.batch(64)
    dataset = dataset.prefetch(AUTO)
    return dataset

# Results

In [ ]:
def pred_TTA(model, image):
    p = np.squeeze(model.predict(image))
    #p += np.squeeze(tf.image.flip_left_right(model.predict(tf.image.flip_left_right(image))))
    #p += np.squeeze(tf.image.flip_up_down(model.predict(tf.image.flip_up_down(image))))
    p += np.squeeze(tf.image.flip_left_right(tf.image.flip_up_down(model.predict(tf.image.flip_up_down(tf.image.flip_left_right(image))))))
    return p / 2 #4

In [ ]:
p = pathlib.Path('../input/hubmap-kidney-segmentation')
subm = {}

for i, filename in tqdm(enumerate(p.glob('test/*.tiff')), 
                        total = len(list(p.glob('test/*.tiff')))):
    
    print(f'{i+1} Predicting {filename.stem}')
    
    dataset = rasterio.open(filename.as_posix(), transform = identity)
    preds = np.zeros(dataset.shape, dtype=np.uint8)    
    
    if SUBMISSION_MODE == 'PUBLIC_TFREC' and MIN_OVERLAP == 300 and WINDOW == 1024 and NEW_SIZE == 512:
        print('SUBMISSION_MODE: PUBLIC_TFREC')
        fnames = glob.glob('/kaggle/input/hubmap-tfrecords-1024-512-test/test/'+filename.stem+'*.tfrec')
        
        if len(fnames)>0: # PUBLIC TEST SET
            for FILENAME in fnames:
                pred = None
                for fold_model in fold_models:
                    tmp = fold_model.predict(get_dataset(FILENAME))/len(fold_models)
                    if pred is None:
                        pred = tmp
                    else:
                        pred += tmp
                    del tmp
                    gc.collect()

                pred = tf.cast((tf.image.resize(pred, (WINDOW,WINDOW)) > THRESHOLD),tf.bool).numpy().squeeze()

                idx = 0
                for img, X1, Y1 in get_dataset(FILENAME):
                    for fi in range(X1.shape[0]):
                        x1 = X1[fi].numpy()
                        y1 = Y1[fi].numpy()
                        preds[x1:(x1+WINDOW),y1:(y1+WINDOW)] += pred[idx]
                        idx += 1
                        
        else: # IGNORE PRIVATE TEST SET (CREATE TFRECORDS IN FUTURE)
            pass
    else:
        print('SUBMISSION_MODE: FULL')
        slices, pos = make_grid(dataset.shape, window=WINDOW, min_overlap=MIN_OVERLAP)
        X1 = pos[0][1:,]
        X2 = pos[1][:-1,]
        overlap_x = X2 - X1
        Y1 = pos[2][1:,]
        Y2 = pos[3][:-1,]
        overlap_y = Y2 - Y1
        preds = np.zeros(dataset.shape, dtype=np.float16)

        if dataset.count != 3:
            print('Image file with subdatasets as channels')
            layers = [rasterio.open(subd) for subd in dataset.subdatasets]
            
        for (x1,x2,y1,y2) in slices:
            if dataset.count == 3:
                image = dataset.read([1,2,3], window=Window.from_slices((x1,x2),(y1,y2)))
                image = np.moveaxis(image, 0, -1)
            else:
                image = np.zeros((WINDOW, WINDOW, 3), dtype=np.uint8)
                for fl in range(3):
                    image[:,:,fl] = layers[fl].read(window=Window.from_slices((x1,x2),(y1,y2)))
                    
            image = cv2.resize(image, (NEW_SIZE, NEW_SIZE),interpolation = cv2.INTER_AREA)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            image = np.expand_dims(image, 0)
            pred = None
            for fold_model in fold_models:
                if pred is None:
                    pred = pred_TTA(fold_model, image)
                else:
                    pred += pred_TTA(fold_model, image)
            pred = pred/len(fold_models)
            pred = cv2.resize(pred, (WINDOW, WINDOW))
            # probability mask for overlap
            mask = np.ones([WINDOW,WINDOW])
            if x1 > 0:
                overlap = overlap_x[np.where(X1 == x1)[0][0]]
                mask[:overlap,:] = mask[overlap,:] / 2
            if x2 < dataset.shape[0]:
                overlap = overlap_x[np.where(X2 == x2)[0][0]]
                mask[-overlap:,:] = mask[-overlap:,:] / 2
            if y1 > 0:
                overlap = overlap_y[np.where(Y1 == y1)[0][0]]
                mask[:,:overlap] = mask[:,:overlap] / 2
            if y2 < dataset.shape[1]:
                overlap = overlap_y[np.where(Y2 == y2)[0][0]]
                mask[:,-overlap:] = mask[:,-overlap:] / 2
            preds[x1:x2,y1:y2] += (mask * pred).astype(np.float16)

    preds = (preds > THRESHOLD).astype(np.uint8)
    subm[i] = {'id':filename.stem, 'predicted': rle_encode_less_memory(preds)}
    if CHECKSUM:
        print('Checksum: '+ str(np.sum(preds)))
    del preds
    gc.collect();

# Making submission

In [ ]:
submission = pd.DataFrame.from_dict(subm, orient='index')
submission.to_csv('submission.csv', index=False)
submission.head()